### Training the UNet model on the original dataset


In [1]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, UpSampling2D, concatenate, Activation, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

2025-01-21 17:43:34.475001: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 17:43:34.492357: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Seeding

In [2]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)


### Hyperparameters

In [3]:
batch_size = 8
lr = 1e-4 ## 0.0001
epochs = 100
height = 768
width = 512

### Path

In [4]:
dataset_path = os.path.join("/home/ahsan/University/Thesis/UNet_Directory/Datasets/segmentation_dataset_path", "aug")
files_dir = os.path.join("/home/ahsan/University/Thesis/UNet_Directory/Datasets/files", "aug")

model_file = os.path.join(files_dir, "unet-aug.h5")
log_file = os.path.join(files_dir, "log-aug.csv")



### Creating folder

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [13]:
create_dir(files_dir)

### UNET (same code as implementation notebook)


#### Convolutional Block

In [6]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

#### Encoder Block

In [7]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

### Decoder Block

In [8]:
def decoder_block(inputs, skip, num_filters):
  x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
  x = Concatenate()([x, skip])
  x = conv_block(x, num_filters)
  return x

### UNet Model

In [9]:
def build_unet(input_shape):
  inputs = Input(input_shape)

  # Encoder
  s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  # Bridge
  b1 = conv_block(p4, 1024)
  # print(s1.shape, s2.shape, s3.shape, s4.shape)

  #Decoder
  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
  # print(outputs.shape)
  model = Model(inputs, outputs, name="UNET")
  return model

### Dataset Pipeline

#### Loading the training and validation dataset

In [10]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train", "image", "*")))
    train_y = sorted(glob(os.path.join(path, "train", "mask", "*")))
    
    valid_x = sorted(glob(os.path.join(path, "valid", "image", "*")))
    valid_y = sorted(glob(os.path.join(path, "valid", "mask", "*")))
    return (train_x, train_y), (valid_x, valid_y)

#### Reading the images

In [11]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = x/255.0
    return x

#### Reading the masks

In [12]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x


#### tf.data pipeline

In [13]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])
    return x, y

In [14]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


### Training

In [15]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

Train: 0 - 0
Valid: 0 - 0


In [16]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [17]:
for x, y in valid_dataset:
    print(x.shape, y.shape)

2025-01-21 17:44:28.909534: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [18]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

NameError: name 'Input' is not defined

In [ ]:
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["acc"])

In [ ]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False),
]

model.fit(train_dataset,
          validation_data=valid_dataset,
          epochs=epochs,
          callbacks=callbacks)

In [ ]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    callbacks=callbacks
)